<a href="https://colab.research.google.com/github/PyCharger/GBC_homeworks/blob/Homeworks/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
import string
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords


def remove_html_tags(text):
    for p in string.punctuation:
      if p in text:
          text = text.replace(p, '')
    return text
def lt_text(text):
    stop_words = set(stopwords.words('russian'))
    stemmer = SnowballStemmer("russian")
    tokens = nltk.word_tokenize(text)
    lemmatized_words = ' '.join([stemmer.stem(word) for word in tokens if word not in stop_words])
    return lemmatized_words

df = pd.read_excel("dataset.xlsx")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df['text']

0        СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1        Пусть это побудет здесь БорьбаВпрямомЭфире How...
2        Раздача пиздюлей от Мунсунга. HowToFtokenoid Б...
3        Не знаю как вам но мне стилистика нравится пус...
4        Скриншоты из новой главы. Тэхунчика показали и...
                               ...                        
53193    8 битная буря снова накрыла пикселями автомоби...
53194    Ира Сидоркова объясняет как сказалась на ее ма...
53195    24 я ракетка мира хорват Марин Чилич обыграл и...
53196    Стал известен календарь мужской сборной России...
53197    Первенство ВХЛ. Первый этап Динамо Алтай Бн ЦС...
Name: text, Length: 53198, dtype: object

In [4]:
df.head()

,oid,text,category
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,esport
1,749208109,Пусть это побудет здесь БорьбаВпрямомЭфире How...,esport
2,749208109,Раздача пиздюлей от Мунсунга. HowToFtokenoid Б...,esport
3,749208109,Не знаю как вам но мне стилистика нравится пус...,esport
4,749208109,Скриншоты из новой главы. Тэхунчика показали и...,esport


In [5]:
df['text'] = df['text'].apply(remove_html_tags)
df['text'] = df.text.str.lower()

In [6]:
df['text'] = df['text'].apply(lt_text)

In [7]:
df['text']

0        спочн сообществ прода 1300р покупк пиш лс 33 г...
1         пуст эт побудет борьбавпрямомэфир howtohtokenoid
2        раздач пиздюл мунсунг howtoftokenoid борьбавпр...
3        зна стилистик нрав пуска стран howtoftokenoid ...
4                скриншот нов глав тэхунчик показа хобин ️
                               ...                        
53193    8 битн бур снов накр пиксел автомобил нужда эк...
53194    ир сидорков объясня сказа машин результат сам ...
53195    24 ракетк мир хорват марин чилич обыгра испанц...
53196    стал извест календар мужск сборн росс розыгрыш...
53197    первенств вхл перв этап динам алта бн цск ввс ...
Name: text, Length: 53198, dtype: object

In [8]:
df.groupby("category").count()

,oid,text
category,,
athletics,4782,4782
autosport,3152,3152
basketball,4329,4329
boardgames,4551,4551
esport,4509,4509
extreme,3172,3172
football,4262,4262
hockey,1851,1851
martial_arts,4259,4259


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


In [ ]:
list_corpus = df['text']
list_labels = df['category']

X_train, X_test, y_train, y_test = train_test_split(list_corpus,
                                                    list_labels,
                                                    test_size=0.2,
                                                    random_state=40)

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
models = [
    RandomForestClassifier(n_estimators=100, random_state=42),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=42)
]
CV = 4
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, X_train_tfidf,y_train, scoring="accuracy", cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=["model_name", "fold_idx", "accuracy"])